In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter("ignore")

from pathlib import Path
import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters, add_genx_model_tags
from powergenome.GenX import reduce_time_domain
from powergenome.load_profiles import make_final_load_curves
from powergenome.params import DATA_PATHS
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
)
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

from powergenome.load_profiles import (
    make_load_curves, 
    add_load_growth, 
    make_final_load_curves, 
    make_distributed_gen_profiles,
)
from powergenome.external_data import make_demand_response_profiles
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    reverse_dict_of_lists,
    remove_feb_29
)

from powergenome.load_profiles import make_final_load_curves
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    reverse_dict_of_lists
)

from powergenome.GenX import reduce_time_domain, add_misc_gen_values
from powergenome.external_data import make_generator_variability

from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.util import init_pudl_connection, load_settings

pd.options.display.max_columns = 200

In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt

In [ ]:
example_system = 0
if example_system ==1:
    run_folder = "example_system" 
    settings_file = "test_settings_mod.yml" 
    scenario = "p1"
else:
    run_folder = "US_Regional"
    settings_file = "Temoa_settings.yml"
    scenario = "p6"

In [ ]:
# Create New SQL File
emptydB = '/Users/aranyavenkatesh/Documents/EnergyOutlook/temoa/data_files/US_National.sqlite'
#outfilename_w_ext = outFilename + '.sqlite'
if example_system==1:
    outputdB = '/Users/aranyavenkatesh/Documents/EnergyOutlook/temoa/data_files/CA_twozone_test.sqlite'
else:
    outputdB = '/Users/aranyavenkatesh/Documents/EnergyOutlook/temoa/data_files/US_Regional_push.sqlite'

In [ ]:
pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd / run_folder / settings_file
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

In [ ]:
all_periods = list(scenario_settings.keys())
start_year = all_periods[0]
run_new = 1
file_prefix = str(settings_path).replace('.yml','_')

In [ ]:
if run_new==1:
    new_gen = pd.DataFrame()
    for year in all_periods[1:]: #new gen for start year is included in all_gen
        gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[year][scenario])
        new_gen_year = gc.create_new_generators()
        new_gen_year.loc[:,'operating_year'] = year
        new_gen = pd.concat([new_gen, new_gen_year]) #create new generators for periods beyond the first
    load_curves = make_final_load_curves(pudl_engine, scenario_settings[start_year][scenario])



    gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[start_year][scenario])
    all_gens = gc.create_all_generators() #create existing and new generators for the first time period

    #add misc_values from misc_gen_inputs_fn file in extra_inputs folder
    all_gens = add_misc_gen_values(all_gens,settings)
    new_gen = add_misc_gen_values(new_gen,settings)

    gen_variability = make_generator_variability(all_gens)

    (
        reduced_resource_profile,
        reduced_load_profile,
        long_duration_storage,
    ) = reduce_time_domain(gen_variability, load_curves, scenario_settings[start_year][scenario])

    if len(settings['region_aggregations'])>1:
        transmission = agg_transmission_constraints(pudl_engine=pudl_engine, settings=settings)
        model_regions_gdf = load_ipm_shapefile(settings)
        transmission = transmission_line_distance(
            trans_constraints_df=transmission,
            ipm_shapefile=model_regions_gdf,
            settings=settings,
        )
        transmission = network_line_loss(transmission=transmission, settings=settings)
        transmission = network_reinforcement_cost(transmission=transmission, settings=settings)
        transmission = network_max_reinforcement(transmission=transmission, settings=settings)
        transmission.to_csv(file_prefix + 'transmission.csv', index=False)

    gen_variability.to_csv(file_prefix + 'gen_variability.csv', index=False)
    load_curves.to_csv(file_prefix + 'load_curves.csv', index=False)
    reduced_load_profile.to_csv(file_prefix + 'reduced_load_profile.csv', index=False)
    reduced_resource_profile.to_csv(file_prefix + 'reduced_resource_profile.csv', index=False)
    all_gens.to_csv(file_prefix + 'all_gens.csv', index=False)
    new_gen.to_csv(file_prefix + 'new_gen.csv', index=False)

In [ ]:
if run_new!=1:
    #reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile.csv')
    #reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_profile.csv')
    all_gens = pd.read_csv(file_prefix + 'all_gens.csv')
    new_gen = pd.read_csv(file_prefix + 'new_gen.csv')
    
    gen_variability = pd.read_csv(file_prefix + 'gen_variability.csv')
    load_curves = pd.read_csv(file_prefix + 'load_curves.csv')
    
    
    (
        reduced_resource_profile,
        reduced_load_profile,
        long_duration_storage,
    ) = reduce_time_domain(gen_variability, load_curves, scenario_settings[start_year][scenario])
    
    if len(settings['region_aggregations'])>1:
        transmission = pd.read_csv(file_prefix + 'transmission.csv')

In [ ]:
map_tech_names = dict()
map_tech_names['biomass'] = 'E_BIO_R'
map_tech_names['conventional_hydroelectric'] = 'E_HYDCONV_R'
map_tech_names['conventional_steam_coal'] = 'E_COALSTM_R'
map_tech_names['natural_gas_fired_combined_cycle'] = 'E_NGACC_R'
map_tech_names['natural_gas_fired_combustion_turbine'] = 'E_NGACT_R'
map_tech_names['natural_gas_steam_turbine'] = 'E_NGASTM_R'
map_tech_names['nuclear'] = 'E_URNLWR_R'
map_tech_names['onshore_wind_turbine'] = 'E_WND_R'
map_tech_names['small_hydroelectric'] = 'E_HYDSM_R'
map_tech_names['solar_photovoltaic'] = 'E_SOLPV_R'
map_tech_names['hydroelectric_pumped_storage'] = 'E_HYDPS_R'
map_tech_names['geothermal'] = 'E_GEO_R'
map_tech_names['naturalgas_ccccsavgcf_mid'] = 'E_NGACC_CCS_N'
map_tech_names['naturalgas_ccavgcf_mid'] = 'E_NGAACC_N'
map_tech_names['naturalgas_ctavgcf_mid'] = 'E_NGAACT_N'
map_tech_names['landbasedwind_ltrg1_mid_110'] = 'E_WND_N'
map_tech_names['utilitypv_losangeles_mid_250_0_2'] = 'E_SOLPVCEN_N'
map_tech_names['naturalgas_ccs100_mid'] = 'E_NGACC_CCS_ZERO_N'
map_tech_names['nuclear_mid'] = 'E_URNLWR_N'
map_tech_names['battery_mid'] = 'E_Batt'
map_tech_names['ev_load_shifting'] = 'E_LD_SHFT'
map_tech_names['offshorewind_otrg10_mid_floating_1_250'] = 'E_OFWND_N'
map_tech_names['geothermal_hydroflash_mid'] = 'E_GEOF_N'
map_tech_names['geothermal_hydrobinary_mid'] = 'E_GEOB_N'
map_tech_names['csp_class5_10hourstes'] = 'E_SOLTHCEN_N'
map_tech_names['biopower_dedicated_mid'] = 'E_BIO_N'
map_tech_names['coal_igccavgcf_mid'] = 'E_COALIGCC_N'
map_tech_names['coal_ccs90avgcf_mid'] = 'E_COALUSC_90CCS_N'

In [ ]:
map_tech_desc = dict()
map_tech_desc['E_BIO_R'] = '#existing bio-energy'
map_tech_desc['E_BIO_N'] = '#new bio-energy'
map_tech_desc['E_HYDSM_R'] = '#existing small hydroelectric power plant'
map_tech_desc['E_HYDPS_R'] = '#existing pumped hydro storage'
map_tech_desc['E_WND_N'] = '#new wind power plant'
map_tech_desc['E_NGACC_CCS_ZERO_N'] = '#new natural gas combined cycle with 100% CCS power plant'
map_tech_desc['E_OFWND_N'] = '#new offshore wind, floating'
map_tech_desc['E_GEOF_N'] = '#new geothermal, hydro flash'
map_tech_desc['E_GEOB_N'] = '#new geothermal, hydro binary'
map_tech_desc['E_COALUSC_90CCS_N'] = '#new ultrasupercritical pulverized coal with 90% CCS power plant'

In [ ]:
#map PG technology names to OEO names
all_gens['Resource'] = all_gens['Resource'].map(map_tech_names)
new_gen['Resource'] = new_gen['Resource'].map(map_tech_names)

In [ ]:
all_gens[all_gens.technology.str.contains('Geo')]

In [ ]:
#multiply capex values by the regional multipliers, since this is not done in PowerGenome
new_gen['capex'] *= new_gen['regional_cost_multiplier']
all_gens['capex'] *= all_gens['regional_cost_multiplier']

In [ ]:
conn = sqlite3.connect(emptydB)
c = conn.cursor()
tech_table = pd.read_sql_query("SELECT * FROM technologies", conn)
tech_table.rename(columns={'tech':'tech_technologies'}, inplace=True)
conn.close()

In [ ]:
cluster_count = all_gens[['technology', 'cluster']].groupby('technology').max().reset_index()
cluster_count.columns = ['technology', 'max_cluster']
all_gens = all_gens.merge(cluster_count, on='technology', how='left')

mask = all_gens['max_cluster']<=1
all_gens.loc[mask, 'tech'] =  all_gens.loc[mask, 'Resource'] + '-' + all_gens.loc[mask,'region']
all_gens.loc[~mask, 'tech'] =  all_gens.loc[~mask, 'Resource'] + '-' + all_gens.loc[~mask,'region'] + \
'-' +all_gens.loc[~mask,'cluster'].map(int).map(str)

In [ ]:
cluster_count = new_gen[['technology', 'cluster']].groupby('technology').max().reset_index()
cluster_count.columns = ['technology', 'max_cluster']
new_gen = new_gen.merge(cluster_count, on='technology', how='left')

mask = new_gen['max_cluster']<=1
new_gen.loc[mask, 'tech'] =  new_gen.loc[mask, 'Resource'] + '-' + new_gen.loc[mask,'region']
new_gen.loc[~mask, 'tech'] =  new_gen.loc[~mask, 'Resource'] + '-' + new_gen.loc[~mask,'region'] + \
'-' +new_gen.loc[~mask,'cluster'].map(int).map(str)

In [ ]:
#concat start year gens with remaining new gens
all_gens_multi_year = pd.concat([all_gens, new_gen])

all_gens_multi_year = all_gens_multi_year.merge(tech_table, left_on = ['Resource'], right_on =['tech_technologies'], how='left')
all_gens_multi_year['flag'].fillna('p', inplace=True)
all_gens_multi_year['sector'].fillna('electric', inplace=True)
all_gens_multi_year['tech_desc'].fillna(all_gens_multi_year['Resource'].map(map_tech_desc), inplace=True)

all_gens_multi_year.loc[all_gens_multi_year['Resource']=='E_HYDPS_R','flag'] = 'ps'

#all_gens_multi_year = all_gens_multi_year[all_gens.columns]
all_gens_multi_year.loc[np.isnan(all_gens_multi_year.operating_year),'operating_year'] = all_periods[0]
all_gens_multi_year.loc[all_gens_multi_year.operating_year==0,'operating_year'] = all_periods[0]

In [ ]:
#account for storage (batteries) capex and FOM for a specific storage duration
stor_dur = 4
mask = all_gens_multi_year['tech'].str.contains('Batt')
orig_batt = all_gens_multi_year.loc[mask,:].copy()
orig_batt['Heat_rate_MMBTU_per_MWh'] = 3412/0.85 #assuming 85% efficiency
all_gens_multi_year.loc[mask,'capex'] += stor_dur*all_gens_multi_year.loc[mask,'capex_mwh']
all_gens_multi_year.loc[mask,'Fixed_OM_cost_per_MWyr'] = 2.5/100*all_gens_multi_year.loc[mask,'capex_mwh']
stor_dur = 8
orig_batt.loc[mask,'capex'] += stor_dur*orig_batt.loc[mask,'capex_mwh']
orig_batt.loc[mask,'Fixed_OM_cost_per_MWyr'] = 2.5/100*orig_batt.loc[mask,'capex_mwh']
orig_batt['Resource'] = 'E_Batt8hr'
orig_batt['tech'] = orig_batt['tech'].str.replace('E_Batt','E_Batt8hr')
orig_batt['tech_technologies'] = orig_batt['tech_technologies'].str.replace('E_Batt','E_Batt8hr')
orig_batt['tech_desc'] = orig_batt['tech_desc'].str.replace('storage','storage (8 hour)')

all_gens_multi_year = pd.concat([all_gens_multi_year, orig_batt])

In [ ]:
def return_region(df_tech):
    return [x[1] for x in df_tech.str.split('-')]

In [ ]:
def return_tech(df_tech):
    output_vals = []
    for val in df_tech:
        try:
            o_val = val.split('-')[0] + '_' + val.split('-')[2]
        except:
            o_val = val.split('-')[0]
        output_vals.append(o_val)
    return output_vals

In [ ]:
def write_sql(df_table, sqlite_table, outputdB):
#code snippets from https://github.com/EnergyModels/temoatools/tree/master/temoatools

    df_table = pd.DataFrame(df_table)
    
    # Set-up sqlite connection
    conn = sqlite3.connect(outputdB)
    c = conn.cursor()

    #----------
    # sqlite file prep
    #----------

    # Create SQL command based on number of entries
    command = 'INSERT INTO ' + sqlite_table + ' VALUES (?'
    for i in range(len(df_table.columns)-1):
        command = command + ',?'
    command = command + ')'

    # Execute SQL command
    try:
        c.executemany(command,np.array(df_table))
    except:
        print(command)
        print(np.array(df_table))
        c.executemany(command, np.array(df_table))

    #----------
    # Save(commit) the changes and close sqlite file
    #----------
    conn.commit()
    conn.close()

In [ ]:

regions_list = all_gens_multi_year.region.unique()

# Delete old *.sqlite file (if it already exists) and copy/rename copy of temoa_schema.sqlite
if os.path.isfile(outputdB):
    os.remove(outputdB)
shutil.copyfile(emptydB, outputdB)
##remove data from tables
conn = sqlite3.connect(outputdB)
c = conn.cursor()

table_list = c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'Output%'").fetchall()
c.execute("UPDATE Efficiency SET tech = TRIM(tech);") #trim spaces. Need to trim carriage return

#remove entire table
for table in ['CapacityToActivity','SegFrac', 'DemandSpecificDistribution', #'LifetimeProcess', 
              'CapacityCredit', 'CapacityFactorTech', 'MinGenGroupWeight','MinGenGroupTarget', #'MinCapacity','StorageDuration',
              'GrowthRateMax','GrowthRateSeed', #'MinActivity',
              'time_of_day', 'time_season', 'time_periods', 'tech_nonrenewable', 'tech_renewable',
              'Output_CapacityByPeriodAndTech','Output_V_Capacity','Output_VFlow_In', 'Output_VFlow_Out',
              'Output_Objective', 'Output_Emissions', 'Output_curtailment', 'Output_Costs',
              'EmissionLimit'
              ]:
    query = """DELETE FROM """ + table
    c.execute(query)

#delete distributed generation (from solar) from Efficiency table in original database
query = """DELETE FROM Efficiency WHERE input_comm='ELCDIST_R'"""
c.execute(query)
query = """DELETE FROM Efficiency WHERE tech='IMPELC'"""
c.execute(query)

techs_keep = ['E_BECCS_N','H2_STO150', 'E_H2CC_N']
for tech_keep in techs_keep:
    #modify the technologies that are retained from US_National for multiple regions
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'tech' in df_cols:
            df = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE tech='" + tech_keep + "'", conn)
            if table[0]=='CapacityFactorTech':
                query = "DELETE FROM " + table[0] + " WHERE tech='" + tech_keep + "'"
                c.execute(query)
            elif (len(df)>0) & ('regions' in df.columns):
                query = "DELETE FROM " + table[0] + " WHERE tech='" + tech_keep + "'"
                c.execute(query)
                for reg in regions_list:
                    df_new = df.copy()
                    df_new['regions'] = reg                        
                    df_new.to_sql(table[0],conn, if_exists='append', index=False)

#modify efficiency for processes that meet demands that can only be met by processes of vintage 2017
for table in ['Efficiency','EmissionActivity']:
    query = """UPDATE """ + table + """ SET vintage=2020
    WHERE vintage=2017
    AND regions || input_comm || tech || output_comm NOT IN 
    (
        SELECT DISTINCT
        regions || input_comm || tech || output_comm
        FROM """ + table + """ WHERE vintage = 2020
    )"""
    c.execute(query)

#modify cost tables to account for these processes
for table in ['CostFixed','CostVariable']:
    query= """UPDATE """ + table + """ SET vintage=2020
        WHERE 
          vintage = 2017 
          AND
          regions || tech NOT IN 
        (SELECT DISTINCT
          regions || tech
        FROM """ + table + """ WHERE vintage = 2020)"""
    c.execute(query)

#delete all elec technologies except for those in techs_keep
for table in ['Efficiency', 'ExistingCapacity', 'DiscountRate', 'CostVariable', 'CostFixed', 'CostInvest']:
    query = "DELETE FROM " + table + " WHERE tech IN\
    (SELECT tech FROM technologies WHERE sector='electric')\
    AND tech NOT IN ('" + "','".join(techs_keep) + "')"
    c.execute(query)

#delete efficiencies from 2017 vintages
for table in ['Efficiency','EmissionActivity']:
    query = """DELETE FROM """ + table + """ WHERE vintage=2017"""
    c.execute(query)
    
        
# Delete row from Efficiency if (t,v) retires at the begining of crent period (which is time_periods[i][0])
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM LifetimeProcess WHERE \
             LifetimeProcess.life_process+LifetimeProcess.vintage<=2020) \
             AND vintage IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.life_process+\
             LifetimeProcess.vintage<=2020);")

# Delete row from Efficiency if (t,v) retires at the begining of crent period (which is time_periods[i][0])
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM LifetimeTech WHERE \
             LifetimeTech.life+Efficiency.vintage<=2020) AND \
             vintage NOT IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.tech\
             =Efficiency.tech);")

# If row is not deleted via the last two DELETE commands, it might still be invalid for period
#  time_periods[i][0] since they can have model default lifetime of 40 years. 
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM Efficiency WHERE \
            40+Efficiency.vintage<=2020) AND \
            tech NOT IN (SELECT tech FROM LifetimeTech) AND \
            vintage NOT IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.tech=Efficiency.tech);")


#remove demands, techinput split before 2020
for table in ['Demand','TechInputSplit', 'MinActivity', 'MaxActivity','CostFixed','CostVariable']:
    query = """DELETE FROM """ + table +""" where periods < 2020"""
    c.execute(query)
for table in ['CostFixed','CostVariable']:
    query = """DELETE FROM """ + table +""" where vintage=2017"""
    c.execute(query)
for table in ['CostInvest', 'DiscountRate']:
    query = """DELETE FROM """ + table + """ where vintage < 2020"""
    c.execute(query)

conn.commit()
conn.close()


In [ ]:
def create_table(outputdB, table_name, primary_key, foreign_key, key_val, references=0):
    create_table_sql = """CREATE TABLE IF NOT EXISTS """ + table_name + """ ( 
                         """ + primary_key + """ text primary key"""
    
    for key in key_val.keys():
        create_table_sql += """, """ + key + """ """ + key_val[key] 
                         
    if references!=0:
        for key in references.keys():
            create_table_sql += """, FOREIGN KEY(""" + foreign_key + """) REFERENCES """ + key + """(""" + references[key] + """)"""
            
        create_table_sql += """);"""
    else:
        create_table_sql += """);"""
    
    conn = sqlite3.connect(outputdB)
    c = conn.cursor()
    c.execute(create_table_sql)
    conn.commit()    

In [ ]:
def create_table_mult_pkey(outputdB, table_name, primary_key, foreign_key, key_val, references=0):
    create_table_sql = """CREATE TABLE IF NOT EXISTS """ + table_name + """("""
    
    for key in key_val.keys():
        create_table_sql +=  key + """ """ + key_val[key] + """, """
                         
    if references!=0:
        for key in references.keys():
            create_table_sql += """ FOREIGN KEY(""" + foreign_key + """) REFERENCES """ + key + """(""" + references[key] + """)"""
            
        #create_table_sql += ""","""
    #else:
        create_table_sql += ""","""
        
    create_table_sql+= """ PRIMARY KEY """ + primary_key  + """);"""
    
    conn = sqlite3.connect(outputdB)
    c = conn.cursor()
    c.execute(create_table_sql)
    conn.commit()

In [ ]:
def swap_regions(x):
    return x.split('-')[1] + '-' + x.split('-')[0]

In [ ]:
#def prep_tables(period, folder_name, gen_data, capfac_variability_data):
    #PREPARE DATABASE TABLES
    
df_gen= all_gens_multi_year
df_gen.rename(columns={'region':'regions'}, inplace=True)

df_gen.loc[:,'regions'] = df_gen.loc[:,'regions'].str.replace('US_N','US')
#df_gen.loc[:, 'tech'] =  df_gen.loc[:, 'Resource'] + '-' + df_gen.loc[:,'regions'] + '-' +df_gen.loc[:,'cluster'].map(int).map(str)

#remove battery, pumped storage and ev_shifting (after column names have been assigned to capfac dataframe)
df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('E_LD_SHFT'),:]
#df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('battery'),:]
#df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('pumped'),:]
#rename operating_year as vintage
df_gen.rename(columns={'operating_year':'vintage'},inplace=True)
df_gen= df_gen.astype({'vintage':int})
df_gen.loc[:,'renewable_nonstor'] = df_gen.loc[:,'RPS'] | df_gen.loc[:,'HYDRO'] 

#capacityfactortech
df_capfac= reduced_resource_profile.reset_index(drop=True)
df_capfac.columns = all_gens.loc[:,'tech'] #rename columns to match df_gen technologies
#remove battery and ev_shifting
df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('E_LD_SHFT')], inplace=True)
#df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('battery')], inplace=True)
#df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('pumped')], inplace=True)

intra_annual_periods = len(df_capfac)
days = intra_annual_periods/24
iter_val = 0
for day in np.arange(days)+1:
    df_capfac.loc[iter_val:iter_val+23,'season_name'] = 'S' + str(int(day))
    df_capfac.loc[iter_val:iter_val+23,'time_of_day_name'] = ['H' + str(int(x)) for x in np.arange(1,25)]
    iter_val+=24
df_capfac = df_capfac.melt(id_vars=['season_name','time_of_day_name'])
df_capfac = df_capfac.rename(columns={'variable':'tech', 'value':'cf_tech'})
df_capfac.loc[:,'cf_tech_notes'] = 'from Power Genome: ' + scenario
df_capfac.insert(0,'regions',return_region(df_capfac.loc[:,'tech']))
#df_capfac.loc[:,'regions'] = df_capfac.loc[:,'regions'].str.replace('US_N','US')
df_capfac.loc[:,'tech'] = return_tech(df_capfac.loc[:,'tech'])
#remove techs that have constant capacity factor
df_sum = df_capfac.groupby(by=['regions','tech']).sum().reset_index()
df_fixed_capfac = df_sum[df_sum.cf_tech==intra_annual_periods]
df_fixed_capfac.rename(columns={'cf_tech': 'cf_tech_max'}, inplace=True)
df_capfac = df_capfac.merge(df_fixed_capfac, on = ['regions','tech'], how='outer')
df_capfac = df_capfac[df_capfac.cf_tech_max!=intra_annual_periods]
df_capfac = df_capfac.drop('cf_tech_max',axis=1)

#segfrac for a chronological model has equal weights for all time periods, with a sum of 1
df_segfrac = df_capfac.loc[:,['season_name','time_of_day_name']].copy()
df_segfrac.drop_duplicates(inplace=True) 
df_segfrac.loc[:,'segfrac'] = 1/len(df_segfrac)
df_segfrac.loc[:,'segfrac_notes'] = 'from Power Genome: ' + scenario

#identify renewables that qualify for RPS and hydro, but not storage 
df_gen.loc[:,'renewable_nonstor'] = df_gen.loc[:,'RPS'] | df_gen.loc[:,'HYDRO'] 

#update lifetime as difference between retirement year and operating year
df_gen.loc[:,'lifetime_diff'] = df_gen.loc[:,'retirement_year'] - df_gen.loc[:,'vintage']
df_gen.loc[:,'lifetime_diff'].fillna(0, inplace=True)

df_gen.loc[:,'lifetime'].fillna(0, inplace=True)
df_gen.loc[:,'lifetime'] = df_gen.loc[:, ['lifetime', 'lifetime_diff']].max(axis=1)

#capacitytoactivity for a chronological model, to normalize to annual time periods, assuming min value is hours
df_c2a = df_gen.loc[:,['tech']].drop_duplicates()
#remove battery and ev_shifting
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('ev_'),:]
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('battery'),:]
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('pumped'),:]

df_c2a.loc[:,'c2a'] = 31.536 #8760/(days*24)
df_c2a.loc[:,'c2a_notes'] = 'from Power Genome: ' + scenario
df_c2a.insert(0,'regions',return_region(df_c2a.loc[:,'tech']))
df_c2a.loc[:,'regions'] = df_c2a.loc[:,'regions'].str.replace('US_N','US')
df_c2a.loc[:,'tech'] = return_tech(df_c2a.loc[:,'tech'])

#investment, fixed and variable costs
df_costs = df_gen.loc[:,['tech', 'capex', 'Fixed_OM_cost_per_MWyr', 'Var_OM_cost_per_MWh', 'vintage']].copy()
df_costs.loc[:,'cost_invest'] = df_costs.loc[:, 'capex']/(10**3) #$/MW to #$M/GW
df_costs.loc[:,'cost_fixed'] = df_costs.loc[:, 'Fixed_OM_cost_per_MWyr']*(10**3)/(10**6) #$/MWyr to #$M/GW-yr
df_costs.loc[:,'cost_variable'] = df_costs.loc[:, 'Var_OM_cost_per_MWh']*(277777.78)/(10**6) #$/MWh to #$M/PJ
df_costs.loc[:,'cost_invest_units'] = '$M/GW'
df_costs.loc[:,'cost_invest_notes'] = 'from Power Genome: ' + scenario
df_costs.loc[:,'cost_fixed_units'] = '$M/GWyr'
df_costs.loc[:,'cost_fixed_notes'] = 'from Power Genome: ' + scenario
df_costs.loc[:,'cost_variable_units'] = '$M/PJ'
df_costs.loc[:,'cost_variable_notes'] = 'from Power Genome: ' + scenario
df_costs.insert(0,'regions',return_region(df_costs.loc[:,'tech']))
df_costs.loc[:,'tech'] = return_tech(df_costs.loc[:,'tech'])

df_cost_invest = df_costs.loc[:,['regions','tech','vintage','cost_invest','cost_invest_units','cost_invest_notes']].copy()
df_cost_invest = df_cost_invest.loc[df_cost_invest.cost_invest>0]

df_costv = df_costs.loc[:,['regions','tech','vintage','cost_variable','cost_variable_units','cost_variable_notes']].drop_duplicates()
df_cost_variable = pd.DataFrame(np.repeat(df_costv.values, len(all_periods), axis=0), columns= df_costv.columns)
df_cost_variable.insert(1,'periods',int((len(df_cost_variable)/len(all_periods)))*all_periods)
df_cost_variable = df_cost_variable[df_cost_variable.vintage<=df_cost_variable.periods]

df_costf = df_costs.loc[:,['regions','tech','vintage','cost_fixed','cost_fixed_units','cost_fixed_notes']]
df_cost_fixed = pd.DataFrame(np.repeat(df_costf.values, len(all_periods), axis=0), columns= df_costf.columns)
df_cost_fixed.insert(1,'periods',int((len(df_cost_fixed)/len(all_periods)))*all_periods)
df_cost_fixed = df_cost_fixed[df_cost_fixed.vintage<=df_cost_fixed.periods]

#efficiency
df_efficiency = df_gen.loc[:,['Resource','tech', 'Heat_rate_MMBTU_per_MWh','renewable_nonstor', 'vintage', 'flag', 'sector', 'tech_desc', 'tech_category']].copy()
df_efficiency.rename(columns={'Resource':'input_comm'}, inplace=True)
#convert technologies with no heat rate to 100% efficiency, by setting the heat rate to 3412.0/1000 MMBTU/MWh
df_efficiency.loc[(df_efficiency.loc[:,'Heat_rate_MMBTU_per_MWh']==0),'Heat_rate_MMBTU_per_MWh'] = 3412.0/1000
df_efficiency.loc[:, 'output_comm'] = df_efficiency.loc[:,'renewable_nonstor'].apply(lambda x: 'ELCP_Renewables' if x ==1 else 'ELCP')
df_efficiency.loc[:,'efficiency'] = 3412.0/(df_efficiency.loc[:,'Heat_rate_MMBTU_per_MWh']*1000)
df_efficiency.loc[:,'efficiency'].fillna(1.0, inplace=True)
df_efficiency.drop(columns=['Heat_rate_MMBTU_per_MWh','renewable_nonstor'], inplace=True)
df_efficiency.loc[:,'eff_notes'] = 'from Power Genome: ' + scenario
#remove battery and ev_shifting
df_efficiency = df_efficiency.loc[~df_efficiency.loc[:,'tech'].str.contains('ev_'),:]
df_efficiency = df_efficiency.loc[~df_efficiency.loc[:,'tech'].str.contains('battery'),:]
df_efficiency.insert(0,'regions',return_region(df_efficiency.loc[:,'tech']))
df_efficiency.loc[:,'tech'] = return_tech(df_efficiency.loc[:,'tech'])

#ramp up and down fractions
df_ramp = df_gen.loc[(df_gen.loc[:,'Ramp_Up_percentage']>0) | (df_gen.loc[:,'Ramp_Dn_percentage']>0), ['regions','tech', 'Ramp_Up_percentage', 'Ramp_Dn_percentage']].copy()
df_ramp.rename(columns={'Ramp_Up_percentage': 'ramp_up', 'Ramp_Dn_percentage': 'ramp_down'}, inplace=True)
df_ramp.loc[:,'tech'] = return_tech(df_ramp.loc[:,'tech'])
df_ramp = df_ramp[(df_ramp.ramp_up<1) | (df_ramp.ramp_down<1)]


#lifetime
df_lifetime = df_gen.loc[:,['tech','lifetime']].copy()
df_lifetime.insert(0,'regions',return_region(df_lifetime.loc[:,'tech']))
df_lifetime.loc[:,'tech'] = return_tech(df_lifetime.loc[:,'tech'])

#existing capacity
df_ex_cap = df_gen.loc[:, ['tech','Existing_Cap_MW', 'vintage' ]]
df_ex_cap = df_ex_cap.loc[df_ex_cap.loc[:,'Existing_Cap_MW']>0,:]
df_ex_cap.loc[:,'exist_cap'] = df_ex_cap.loc[:,'Existing_Cap_MW']/1000 #GW
df_ex_cap.drop(columns=['Existing_Cap_MW'], inplace=True)
df_ex_cap.loc[:,'exist_cap_units'] = 'GW'
df_ex_cap.loc[:,'exist_cap_notes'] = 'from Power Genome: ' + scenario
##FIX THIS for zero value years for EV load shifting, removing this row for now
df_ex_cap = df_ex_cap.loc[df_ex_cap.loc[:,'vintage']!=0,:]
df_ex_cap.insert(0,'regions',return_region(df_ex_cap.loc[:,'tech']))
df_ex_cap.loc[:,'tech'] = return_tech(df_ex_cap.loc[:,'tech'])

#discount rate
df_wacc = df_gen.loc[(df_gen.loc[:,'waccnomtech']>0), ['tech', 'vintage', 'waccnomtech']].copy()
df_wacc.rename(columns={'waccnomtech': 'tech_rate'}, inplace=True)
df_wacc.loc[:,'tech_rate_notes'] = 'from Power Genome: ' + scenario
df_wacc.insert(0,'regions',return_region(df_wacc.loc[:,'tech']))
df_wacc.loc[:,'tech'] = return_tech(df_wacc.loc[:,'tech'])

#capital recovery years
df_cap_rec_years = df_gen.loc[(df_gen.loc[:,'cap_recovery_years']>0),['tech', 'cap_recovery_years']].copy()
df_cap_rec_years.rename(columns={'cap_recovery_years': 'loan'}, inplace=True)
df_cap_rec_years.loc[:,'loan_notes'] = 'from Power Genome: ' + scenario
df_cap_rec_years.insert(0,'regions',return_region(df_cap_rec_years.loc[:,'tech']))
df_cap_rec_years.loc[:,'tech'] = return_tech(df_cap_rec_years.loc[:,'tech'])

#lifetime
df_lifetime = df_gen.loc[(df_gen.loc[:,'lifetime']>0),['tech', 'lifetime']].copy()
df_lifetime.rename(columns={'lifetime': 'life'}, inplace=True)
df_lifetime.loc[:,'life_notes'] = 'from Power Genome: ' + scenario
df_lifetime.insert(0,'regions',return_region(df_lifetime.loc[:,'tech']))
df_lifetime.loc[:,'tech'] = return_tech(df_lifetime.loc[:,'tech'])

#capacity credit
df_ccredit = df_gen.loc[df_gen.loc[:,'CapRes']>0, ['tech', 'vintage', 'CapRes']].copy()
df_ccredit  = pd.DataFrame(np.repeat(df_ccredit.values, len(all_periods), axis=0), columns= df_ccredit.columns)
df_ccredit.insert(0,'periods',int((len(df_ccredit)/len(all_periods)))*all_periods)
df_ccredit.rename(columns={'CapRes': 'cf_tech'}, inplace=True)
df_ccredit.loc[:,'cf_tech_notes'] = 'from Power Genome: ' + scenario
df_ccredit.insert(0,'regions',return_region(df_ccredit.loc[:,'tech']))
df_ccredit.loc[:,'tech'] = return_tech(df_ccredit.loc[:,'tech'])

#max capacity, for renewables, primarily
df_maxcap = df_gen.loc[(df_gen.loc[:,'Max_Cap_MW']>0),['tech','Max_Cap_MW','vintage']].copy()
df_maxcap.rename(columns={'Max_Cap_MW':'maxcap', 'vintage':'periods'}, inplace=True)
df_maxcap.loc[:,'maxcap'] = df_maxcap.loc[:,'maxcap']/1000 #convert MW to GW
df_maxcap.loc[:,'maxcap_units'] = 'GW' #convert MW to GW
df_maxcap.loc[:,'maxcap_tech_notes'] = 'from Power Genome: ' + scenario
df_maxcap.insert(0,'regions',return_region(df_maxcap.loc[:,'tech']))
df_maxcap.loc[:,'tech'] = return_tech(df_maxcap.loc[:,'tech'])

#load, estimating demand specific distribution, same distribution for all demands
df_load_i = reduced_load_profile
ind = list(reduced_load_profile.columns).index('Time_index')
df_load_i = reduced_load_profile[reduced_load_profile.columns[ind+1:]]
iter_val = 0
for day in np.arange(days)+1:
    df_load_i.loc[iter_val:iter_val+23,'season_name'] = 'S' + str(int(day))
    df_load_i.loc[iter_val:iter_val+23,'time_of_day_name'] = ['H' + str(int(x)) for x in np.arange(1,25)]
    iter_val+=24
df_load_i = df_load_i.melt(id_vars=['season_name','time_of_day_name'])
df_load_i = df_load_i.rename(columns={'variable':'regions', 'value':'dds_vals'})
df_load_i.loc[:, 'regions'] = df_load_i.loc[:, 'regions'].str.replace('US_N','US')

df_load = pd.DataFrame()
for demn in ['RSC', 'RSH','RLT','ROELC','RWH','CLT','COELC','COEELC','CSC','CSH','CWH']:
    for region in df_load_i.regions.unique():
        df_load_i.loc[(df_load_i.regions==region),'demand_name'] = demn
        df_load_i.loc[(df_load_i.regions==region),'dds'] = df_load_i.loc[(df_load_i.regions==region),'dds_vals']/df_load_i.loc[(df_load_i.regions==region),'dds_vals'].sum()
        df_load = pd.concat([df_load, df_load_i.loc[(df_load_i.regions==region),['regions','season_name','time_of_day_name','demand_name','dds']]])
df_load.loc[:,'dds_notes'] = 'from Power Genome: ' + scenario
df_load.loc[:,'regions'] = df_load.loc[:,'regions'].str.replace('z1','US')

if len(settings['region_aggregations'])>1:

    #transmission efficiency
    df_trans_efficiency = pd.DataFrame()
    df_trans_efficiency.loc[:,'regions']  = transmission.loc[:,'Transmission Path Name'].str.replace('_to_','-')
    df_trans_efficiency.loc[:,'input_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'tech'] = 'E_TRANS'
    df_trans_efficiency.loc[:,'vintage'] = all_periods[0]-1
    df_trans_efficiency.loc[:,'output_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'efficiency'] = 1 - transmission.loc[:,'Line_Loss_Percentage']
    df_trans_efficiency.loc[:,'eff_notes'] = 'from Power Genome: ' + scenario
    df_trans_efficiency.loc[:,'flag'] = 'p'
    df_trans_efficiency.loc[:,'sector'] = 'electric'
    df_trans_efficiency.loc[:,'tech_desc'] = '#electricity transmission'
    
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency])
    df_trans_efficiency['regions'] = df_trans_efficiency['regions'].apply(swap_regions)
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency])

    #transmission costs
    df_trans_c = pd.DataFrame()
    df_trans_c.loc[:,'regions']  = transmission.loc[:,'Transmission Path Name'].str.replace('_to_','-')
    df_trans_c.loc[:,'tech'] = 'E_TRANS'
    df_trans_c.loc[:,'vintage'] = all_periods[0]-1
    df_trans_c.loc[:, 'cost_fixed'] = transmission.loc[:, 'Line_Reinforcement_Cost_per_MW_yr']*(10**3)/(10**6) #$/MWyr to #$M/GW-yr
    df_trans_c.loc[:, 'cost_fixed_units'] = '$M/GWyr'
    df_trans_c.loc[:, 'cost_fixed_notes'] = 'from Power Genome: ' + scenario
    df_trans_costs = pd.DataFrame(np.repeat(df_trans_c.values, len(all_periods), axis=0), columns= df_trans_c.columns)
    df_trans_costs.insert(1,'periods',int((len(df_trans_costs)/len(all_periods)))*all_periods)
    df_trans_costs = df_trans_costs[df_trans_costs.vintage<=df_trans_costs.periods]
    
    df_cost_fixed = pd.concat([df_cost_fixed, df_trans_costs])
    df_trans_costs['regions'] = df_trans_costs['regions'].apply(swap_regions)
    df_cost_fixed = pd.concat([df_cost_fixed, df_trans_costs])

    #trans maximum capacity
    df_trans_cap = pd.DataFrame()

    df_trans_cap.loc[:,'regions']  = transmission.loc[:,'Transmission Path Name'].str.replace('_to_','-')
    df_trans_cap.loc[:,'vintage']  = all_periods[0]-1

    df_trans_cap.loc[:,'exist_cap'] = abs(transmission.loc[:,['Line_Max_Flow_MW', 'Line_Min_Flow_MW']]).max(axis=1).values/1000

    df_trans_cap.insert(1,'tech', 'E_TRANS')
    df_trans_cap.loc[:,'exist_cap_units'] = 'GW'
    df_trans_cap.loc[:,'exist_cap_notes'] = 'from Power Genome: ' + scenario
    
    df_ex_cap = pd.concat([df_ex_cap, df_trans_cap])
    df_trans_cap['regions'] = df_trans_cap['regions'].apply(swap_regions)
    df_ex_cap = pd.concat([df_ex_cap, df_trans_cap])

    

In [ ]:
#add input_comm from commodities list
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('BIO'),'input_comm'] = 'AGR'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('SOL'),'input_comm'] = 'ethos_R'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('WND'),'input_comm'] = 'ethos_R'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('HYD'),'input_comm'] = 'ethos_R'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('GEO'),'input_comm'] = 'ethos_R'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('URN'),'input_comm'] = 'URN_N'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('NG'),'input_comm'] = 'E_NGA'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('COAL'),'input_comm'] = 'COALSTM_N'
df_efficiency.loc[df_efficiency.loc[:,'input_comm'].str.contains('E_Batt'),'input_comm'] = 'ELCP'

In [ ]:
#change existing wind and solar efficiencies to 1
df_efficiency.loc[(df_efficiency.loc[:,'tech'].str.contains('wind')) & (df_efficiency.loc[:,'vintage']<start_year), 'efficiency'] = 1
df_efficiency.loc[(df_efficiency.loc[:,'tech'].str.contains('solar')) & (df_efficiency.loc[:,'vintage']<start_year), 'efficiency'] = 1

In [ ]:
#WRITE TO DATABASE


#CapacityToActivity
df_table = df_c2a.drop_duplicates()
sqlite_table = 'CapacityToActivity'
write_sql(df_table, sqlite_table, outputdB)

#SegFrac
df_table = df_segfrac.drop_duplicates()
sqlite_table = 'SegFrac'
write_sql(df_table, sqlite_table, outputdB)

#CostInvest
df_table = df_cost_invest.copy()
df_table = df_table.loc[df_table.loc[:,'cost_invest']>0,:]
sqlite_table = 'CostInvest'
write_sql(df_table, sqlite_table, outputdB)

#CostVariable
df_table = df_cost_variable.drop_duplicates()
sqlite_table = 'CostVariable'
write_sql(df_table, sqlite_table, outputdB)

#CostFixed
df_table = df_cost_fixed.drop_duplicates()
sqlite_table = 'CostFixed'
write_sql(df_table, sqlite_table, outputdB)

#Efficiency
df_table = df_efficiency[['regions', 'input_comm','tech','vintage','output_comm', 'efficiency', 'eff_notes']].drop_duplicates()
sqlite_table = 'Efficiency'
write_sql(df_table, sqlite_table, outputdB)

#tech_curtailment
df_table = pd.DataFrame(df_efficiency.loc[['wind' in x or 'pv' in x or 'solar' in x for x in df_efficiency.loc[:,'tech']],'tech'].drop_duplicates())
df_table.insert(1,'notes','')
sqlite_table = 'tech_curtailment'
write_sql(df_table, sqlite_table, outputdB)

#tech_nonrenewable
df_table = df_efficiency.loc[df_efficiency.loc[:,'output_comm']=='ELCP','tech'].drop_duplicates()
sqlite_table = 'tech_nonrenewable'
write_sql(df_table, sqlite_table, outputdB)

#tech_renewable
df_table = df_efficiency.loc[df_efficiency.loc[:,'output_comm']=='ELCP_Renewables','tech'].drop_duplicates()
sqlite_table = 'tech_renewable'
write_sql(df_table, sqlite_table, outputdB)

#tech_production in technologies table
df_table = df_efficiency[['tech', 'flag', 'sector', 'tech_desc', 'tech_category']].drop_duplicates()
df_table = df_table[~df_table['tech'].isin(tech_table['tech_technologies'])]
sqlite_table = 'technologies'
write_sql(df_table, sqlite_table, outputdB)

#Ramping
#ramping techs
references = dict()
references['technologies'] = 'tech'
create_table(outputdB, 'tech_ramping', 'tech', 'tech', dict(),references)

df_table = df_ramp.loc[:,['tech']].drop_duplicates()
sqlite_table = 'tech_ramping'
write_sql(df_table, sqlite_table, outputdB)

#ramp up
ramp_up_pairs = dict()
ramp_up_pairs['regions'] = 'text'
ramp_up_pairs['tech'] = 'text'
ramp_up_pairs['ramp_up'] = 'real'
references = dict()
references['technologies'] = 'tech'
create_table_mult_pkey(outputdB, 'RampUp', "(regions, tech)", 'tech', ramp_up_pairs,references)

df_table = df_ramp.loc[:,['regions','tech', 'ramp_up']].drop_duplicates()
sqlite_table = 'RampUp'
write_sql(df_table, sqlite_table, outputdB)

#ramp down
ramp_up_pairs = dict()
ramp_up_pairs['regions'] = 'text'
ramp_up_pairs['tech'] = 'text'
ramp_up_pairs['ramp_up'] = 'real'
references = dict()
references['technologies'] = 'tech'
create_table_mult_pkey(outputdB, 'RampDown', "(regions, tech)", 'tech', ramp_up_pairs,references)

df_table = df_ramp.loc[:,['regions','tech', 'ramp_down']].drop_duplicates()
sqlite_table = 'RampDown'
write_sql(df_table, sqlite_table, outputdB)

#ExistingCapacity
df_table = df_ex_cap.drop_duplicates()
sqlite_table = 'ExistingCapacity'
write_sql(df_table, sqlite_table, outputdB)

#DiscountRate
df_table = df_wacc.loc[:,['regions','tech', 'vintage', 'tech_rate', 'tech_rate_notes']].drop_duplicates()
sqlite_table = 'DiscountRate'
write_sql(df_table, sqlite_table, outputdB)

#LifetimeLoanTech
df_table = df_cap_rec_years.drop_duplicates()
sqlite_table = 'LifetimeLoanTech'
write_sql(df_table, sqlite_table, outputdB)

#LifetimeTech
conn = sqlite3.connect(outputdB)
c = conn.cursor()
df_lifetime_current = pd.read_sql_query("SELECT * FROM LifetimeTech", conn)
nonoverlap_tech = df_lifetime_current[~df_lifetime_current['tech'].isin(df_lifetime['tech'])]
query = "DELETE FROM LifetimeTech"
c.execute(query)
conn.commit()
conn.close()
df_lifetime.loc[:,'life']=120 #all to 120 years for now
df_table = pd.concat([nonoverlap_tech, df_lifetime.drop_duplicates()])
sqlite_table = 'LifetimeTech'
write_sql(df_table, sqlite_table, outputdB)

#CapacityCredit
df_table = df_ccredit.loc[:,['regions','periods','tech', 'vintage','cf_tech','cf_tech_notes']].drop_duplicates()
sqlite_table = 'CapacityCredit'
write_sql(df_table, sqlite_table, outputdB)

#tech_reserve
df_table = df_ccredit.loc[:,['tech','cf_tech_notes']].drop_duplicates(subset='tech')
df_table = df_table[~df_table['tech'].isin(tech_table['tech_technologies'])]
df_table.rename(columns={'cf_tech_notes':'notes'}, inplace=True)
sqlite_table = 'tech_reserve'
write_sql(df_table, sqlite_table, outputdB)

#CapacityFactorTech
df_table = df_capfac.drop_duplicates()
sqlite_table = 'CapacityFactorTech'
write_sql(df_table, sqlite_table, outputdB)

#MaxCapacity
df_table = df_maxcap.loc[:,['regions','periods','tech', 'maxcap','maxcap_units','maxcap_tech_notes']].drop_duplicates()
sqlite_table = 'MaxCapacity'
write_sql(df_table, sqlite_table, outputdB)

#DSD
df_table = df_load.loc[:,['regions','season_name','time_of_day_name','demand_name','dds','dds_notes']]
sqlite_table = 'DemandSpecificDistribution'
write_sql(df_table, sqlite_table, outputdB)

#time_season
df_table = pd.DataFrame(df_capfac.loc[:,'season_name'].drop_duplicates())
df_table.rename(columns={'season_name':'t_season'},inplace=True)
sqlite_table = 'time_season'
write_sql(df_table, sqlite_table, outputdB)

#time_of_day
df_table = pd.DataFrame(df_capfac.loc[:,'time_of_day_name'].drop_duplicates())
df_table.rename(columns={'time_of_day_name':'t_day'},inplace=True)
sqlite_table = 'time_of_day'
write_sql(df_table, sqlite_table, outputdB)

#time_periods
df_table = pd.DataFrame(columns=['t_periods','flag'])
#df_table.loc[:,'t_periods'] = np.sort(df_costs.loc[:,'vintage'].unique().astype(int))
df_table.loc[:,'t_periods'] = np.hstack((np.arange(df_costs.loc[:,'vintage'].astype(int).min()-1,2020), 
                                         np.arange(2020,2056,5)))
df_table.loc[:,'flag'] = ['e' if x <2020  else 'f' for x in df_table.loc[:,'t_periods']]
sqlite_table = 'time_periods'
write_sql(df_table, sqlite_table, outputdB)


if len(settings['region_aggregations'])>1 & ('E_TRANS' in df_efficiency.tech.unique()):
    df_table = pd.DataFrame(columns = ['tech', 'notes'])
    df_table.loc[0] = ['E_TRANS','']
    sqlite_table = 'tech_exchange'
    write_sql(df_table, sqlite_table, outputdB)


In [ ]:
#map regions to descriptions
map_region_desc = dict()
map_region_desc['CA'] = 'California'
map_region_desc['NW'] = 'Northwestern US'
map_region_desc['SW'] =  'Southwestern US'
map_region_desc['TX'] =  'Texas'
map_region_desc['N_CEN'] =  'North Central US'
map_region_desc['CEN'] =  'Central US'
map_region_desc['SE'] =  'Southeastern US'
map_region_desc['MID_AT'] =  'Mid Atlantic US'
map_region_desc['NE'] = 'Northeastern US'

df_regions = pd.DataFrame.from_dict(map_region_desc, orient='index').reset_index()
df_regions.columns = ['regions', 'region_note']

In [ ]:
conn = sqlite3.connect(outputdB)
c = conn.cursor()

iterval = 0
while len(c.execute("SELECT * FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency)\
                       AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');").fetchall()) > 0:

    c.execute("DELETE FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency) \
             AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');")
    iterval+=1
    if iterval>10:
        break
print(iterval)

for table in table_list:
    if table[0] != 'Efficiency':  
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'tech' in df_cols:
            c.execute("UPDATE "+str(table[0])+" SET tech = TRIM(tech, CHAR(37,13,10));")
            try:
                if 'vintage' in df_cols:
                    # If t doesn't exist in Efficiency table after the deletions made above, 
                    # it is deleted from other tables.
                    c.execute("DELETE FROM "+str(table[0])+" WHERE tech || vintage \
                        NOT IN (SELECT tech || vintage FROM Efficiency)")
                else:
                    c.execute("DELETE FROM "+str(table[0])+" WHERE tech NOT IN (SELECT tech FROM Efficiency);")
            except:
                print(table[0])
        if table[0] == 'EmissionActivity':
            c.execute("DELETE FROM EmissionActivity WHERE input_comm || tech || vintage || output_comm \
                        NOT IN (SELECT input_comm || tech || vintage || output_comm FROM Efficiency)")


c.execute("UPDATE commodities SET comm_name = TRIM(comm_name, CHAR(10,13,37))")
    # delete unused commodities otherwise the model throws an error
c.execute("DELETE FROM commodities WHERE flag!='e' AND comm_name NOT IN (SELECT input_comm from Efficiency UNION SELECT output_comm from Efficiency);")

#update regional transportation demands 
df_transport = pd.read_csv('../TransportationSector/NREL_EFS/transportation_regional_demands_adjusted.csv')
df_transport = df_transport[['regions', 'periods', 'demand_comm', 'demand','demand_units','demand_notes']]
query = "DELETE FROM Demand WHERE demand_units LIKE '%vmt%' \
OR demand_units LIKE '%mile%' OR demand_units LIKE '%bpm%'OR demand_units LIKE '%btm%'"
c.execute(query)
df_transport.to_sql('Demand',conn, if_exists='append', index=False)

#update regional building demands 
df_buildings = pd.read_csv('../BuildingsSector/NREL_EFS/buildings_regional_demands_adjusted.csv')
df_buildings = df_buildings[['regions', 'periods', 'demand_comm', 'demand','demand_units','demand_notes']]
query = "DELETE FROM Demand WHERE \
substr(demand_comm,1,1) = 'C'OR substr(demand_comm,1,1) = 'R'"
c.execute(query)
df_buildings.to_sql('Demand',conn, if_exists='append', index=False)

#update regional industry demands 
df_industry = pd.read_csv('../IndustrialSector/NREL_EFS/industry_regional_demands_adjusted.csv')
df_industry = df_industry[['regions', 'periods', 'demand_comm', 'demand','demand_units','demand_notes']]
query = "DELETE FROM Demand  \
WHERE demand_comm LIKE '%IND%'"
c.execute(query)
df_industry.to_sql('Demand',conn, if_exists='append', index=False)

#find remaining demands at US level, divide by 9 to get demands in each region
df_remains = pd.read_sql_query("SELECT * FROM Demand WHERE regions='US'", conn)
query = "DELETE FROM Demand WHERE regions='US'"
c.execute(query)
for reg in df_industry.regions.unique():
    df_remains_new = df_remains.copy()
    df_remains_new.regions=reg
    df_remains_new['demand'] /=9
    df_remains_new.to_sql('Demand',conn, if_exists='append', index=False)

for table in table_list:
    df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
    df_cols = [x[1] for x in df_cols]
    
    if table[0]=='regions':
        query = "DELETE FROM " + table[0]
        c.execute(query)
        df_regions.to_sql(table[0],conn, if_exists='append', index=False)
    elif 'regions' in df_cols:
        df_remains = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE regions='US'", conn)
        query = "DELETE FROM " + table[0] + " WHERE regions='US'"
        c.execute(query)
        for reg in df_industry.regions.unique():
            df_remains_new = df_remains.copy()
            df_remains_new.regions=reg
            if table[0] in ['ExistingCapacity']:
                df_remains_new['exist_cap'] /=9
            #if table[0] in ['GrowthRateMax']:
            #    df_remains_new['growthrate_max'] /=9
            #if table[0] in ['GrowthRateSeed']:
            #    df_remains_new['growthrate_seed'] /=9
            if table[0] in ['MinActivity']:
                df_remains_new['minact'] /=20
            df_remains_new.to_sql(table[0],conn, if_exists='append', index=False)

conn.commit()
conn.close()

In [ ]:
con=sqlite3.connect(outputdB, isolation_level=None)
con.execute("VACUUM")
con.commit()
con.close()

In [ ]:
conn = sqlite3.connect(outputdB)
with open(outputdB.replace('ite',''), 'w') as f:
    for line in conn.iterdump():
        '''
        if 'e+' in line or 'e-0' in line:
            val = [x for x in line.split(',') if 'e+' in x or 'e-0' in x][0]
            ls = line.split(',')
            ind = ls.index(val)
            ls[ind] = [str(round(float(x),2)) for x in line.split(',') if 'e+' in x or 'e-0' in x ][0]
            line = ','.join(ls)
        '''
        if 'CREATE TABLE' in line:
            f.write('\n' )
        f.write('%s\n' % line)
        
conn.close()